# Use Model Service

In [1]:
import os
import sys
import time

In [2]:
PROJECT_FOLDER = "D:\EBUSTOS\DEVELOP\INDRA\model-microservice-example" #"path/to/folder"
simulator_project_folder = os.path.join(PROJECT_FOLDER, "simulation")
model_project_folder = os.path.join(PROJECT_FOLDER, "precipitationmodel")

In [3]:
if PROJECT_FOLDER not in sys.path:
    sys.path.insert(0, PROJECT_FOLDER)

In [4]:
from simulation import Simulator
from model import ModelService
from config import ModelConfig

### Print model info

In [5]:
from config import ModelConfig
print(ModelConfig.SUPERVISOR_ENDPOINT)

None


In [6]:
model = ModelService()

Not starting scheduler for use not supervisor
implemented load model
default loading preprocessor
loading serialized file D:\EBUSTOS\DEVELOP\INDRA\model-microservice-example\data\precipitation_preprocessor.pkl with serializer FileSerializers.PICKLE
Info - IotBrokerClient will be soon deprecated, please use DigitalClient instead
Setted digital client: Client.IotBrokerClient(host=industryonesait.cwbyminsait.com, port=443, _Client__protocol=https, _Client__avoid_ssl_certificate=True, is_connected=False, iot_client=<iot_client>, iot_clientId=<iot_client>:PythonClient, iot_client_token=None, session_key=None)


In [ ]:
print(model.status)

In [ ]:
model.training_score = 3

In [ ]:
model.update_status()

In [ ]:
model.notify_status()

In [ ]:
model.get_task_from_supervisor()

In [ ]:
print("Model name: {}".format(model.name))
print("Model info: {}".format(model.info()))
print("Model sample: {}".format(model.sample()))
print("Model labels: {}".format(model.labels))
print("Model status: {}".format(model.status))

### Prediction with model

In [ ]:
nolluvia = {'tmp0': 27.12, 'tmp1': 10.36, 'hPa': 987, 'hum': 0.23}
lluvia = {'tmp0': 19.45, 'tmp1': 7.10, 'hPa': 952, 'hum': 1.00}
data_Ready = model.transform_in(lluvia)
data_Ready

In [ ]:
res = model.predict(data_Ready)
res

In [ ]:
model.transform_out(res)

### Training model

In [ ]:
model.set_digital_client(
    host="industryonesait.cwbyminsait.com",
    port = 443,
    iot_client="predictionclient", 
    iot_client_token="1b5e4bf62cb94812a838e215b20cdcdd"
)

model.digital_client.protocol = "https"
model.digital_client.avoid_ssl_certificate = True
print(model.digital_client)

In [ ]:
model.is_training = True

In [ ]:
model.start_train(ontology="predictionmodel")

In [ ]:
model.digital_client.leave()

In [ ]:
model.digital_client.is_connected

In [ ]:
model.digital_client.query("predictionmodel", "db.predictionmodel.find().limit(8)", "NATIVE")

In [ ]:
model.train(ontology="predictionmodel", query="db.predictionmodel.find().limit(8)")
model.last_trained

In [ ]:
from config import ModelConfig

In [ ]:
ModelConfig.save_config()

In [ ]:
def display_arguments(func):
    def display_and_call(*args, **kwargs):      
        args = list(args)
        print('must-have arguments are:')
        print(args[0].is_training)
        for i in args:
            print(i)          
        print('optional arguments are:')
        for kw in kwargs.keys():
            print( kw+'='+str( kwargs[kw] ) )          
        return func(*args, **kwargs)   
    return display_and_call

def not_while_training(func):
    def call(*args, **kwargs):      
        args = list(args)
        model_service_class = args[0]
        if isinstance(model_service_class, BaseModelService):
            if model_service_class.is_training:
                raise RuntimeError("Not possible to train service while training")
        else:
            raise RuntimeWarning("Wrapper used in wrong class: {}".format(model_service_class))
         
        return func(*args, **kwargs)   
    return call

In [ ]:
from model import BaseModelService
class M(BaseModelService):
    def __init__(self):
        self.is_training = False
        
    @not_while_training
    def train(self):
        print("Training...")
        return True


In [ ]:
m = M()



In [ ]:
m.train()

In [ ]:
m.is_training = True